In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/smpd3/smpd3_dataset/20180702_MO_Pax7_Cece/wm/migration'

In [6]:
# Add experiment date here to apply to dataframe
expt_date = '20180702'
treatment = 'SMPD3MO'
result_type = 'Migration'

path = os.path.abspath('')+'/pax7migrationCSV/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Date'] = expt_date
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    df['Target'], df['Treatment'], df['Enhancer'], df['Fluor'], df['Embryo'], df['Mag'], df['Somites'] = zip(*df['Image'].map(lambda x: x.split('_')))
    
    list_.append(df)

full_df = pd.concat(list_)
full_df.head()

,,Label,Area,Mean,Angle,IntDen,RawIntDen,Length,Date,Image,...,ROI,delete2,delete3,Target,Treatment,Enhancer,Fluor,Embryo,Mag,Somites
0,1,A:CntlArea:c:1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10...,122618.821,1658.938,0.000,2.034170e+08,9.869056e+08,0.000,20180702,SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss,...,CntlArea,c,1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss.czi #1,SMPD3,MO,TCFLef,RFP,Emb3,10x,9ss
1,2,A:ExptArea:c:1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10...,85467.442,1725.042,0.000,1.474349e+08,7.153008e+08,0.000,20180702,SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss,...,ExptArea,c,1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss.czi #1,SMPD3,MO,TCFLef,RFP,Emb3,10x,9ss
2,3,A:Cntl:c:1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9s...,79.149,1603.868,78.250,1.269438e+05,6.158853e+05,173.894,20180702,SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss,...,Cntl,c,1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss.czi #1,SMPD3,MO,TCFLef,RFP,Emb3,10x,9ss
3,4,A:Expt:c:1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9s...,89.454,1774.416,82.432,1.587292e+05,7.700966e+05,196.478,20180702,SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss,...,Expt,c,1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss.czi #1,SMPD3,MO,TCFLef,RFP,Emb3,10x,9ss
4,5,A:Cntl:c:1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9s...,116.662,1626.337,82.988,1.897312e+05,9.205070e+05,256.613,20180702,SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss,...,Cntl,c,1/3 - SMPD3_MO_TCFLef_RFP_Emb3_10x_9ss.czi #1,SMPD3,MO,TCFLef,RFP,Emb3,10x,9ss


In [7]:
# Mean length values and group
area_meanLength = (full_df.groupby(['Embryo', 'ROI'])['Area', 'Length']).mean()

# Extract Areas, generate ratios
area_cntl = area_meanLength.xs('CntlArea', level='ROI')['Area'] 
area_expt = area_meanLength.xs('ExptArea', level='ROI')['Area'] 

# Generate ratios as Expt/Cntl
area_ratios = pd.DataFrame(area_expt / area_cntl)
area_ratios.columns = ['Expt/Cntl Area']

# Extract lengths, generate ratios
length_cntl = area_meanLength.xs('Cntl', level='ROI')['Length']
length_expt = area_meanLength.xs('Expt', level='ROI')['Length'] 

# Generate ratios as Expt/Cntl
length_ratios = pd.DataFrame(length_expt / length_cntl)
length_ratios.columns = ['Expt/Cntl Length']

In [8]:
# Combine processed values into single dataframe and output as csv file 'Results.csv'
area_cntl = pd.DataFrame(area_cntl)
area_cntl.columns = ['Cntl Area']
area_expt = pd.DataFrame(area_expt)
area_expt.columns = ['Expt Area']

length_cntl = pd.DataFrame(length_cntl)
length_cntl.columns = ['Cntl Length']
length_expt = pd.DataFrame(length_expt)
length_expt.columns = ['Expt Length']


results = pd.concat([area_cntl, area_expt, area_ratios, length_cntl, length_expt, length_ratios], axis=1, sort=True)
results['ID'] = expt_date + '_' + results.index + '_Cece'
results.to_csv(expt_date + '_' + treatment + '_' + result_type + '_results.csv')